In [ ]:
! pip install fastai ipywidgets

In [ ]:
from fastai.vision.all import *
from tqdm import tqdm

In [ ]:
data = get_image_files("./good") + get_image_files("./bad")

In [ ]:
def good_bad_label(x): return x.parent.name

dls = ImageDataLoaders.from_path_func(".", data, good_bad_label, bs=64, item_tfms=RandomResizedCrop(256))

In [ ]:
dls.show_batch()

In [ ]:
learn = vision_learner(
    dls,
    efficientnet_b0,
    metrics=error_rate,
    #cbs=[GradientAccumulation(28)],
).to_fp16()
lrfind = learn.lr_find()


In [ ]:
learn.fine_tune(5, lrfind.valley / 2)

In [ ]:
learn.show_results(max_n=32)

In [ ]:
pics = [p for p in (get_image_files("/home/lsb/DCIMsmol/")) if 0 < p.stat().st_size]
bulk_pic_dl = dls.test_dl(pics)

In [ ]:
preds, targets = learn.get_preds(dl=bulk_pic_dl) # targets = None, because this is a test dataset

In [ ]:
pic_goodness = preds[:,list(learn.dls.vocab).index("good")]
pic_goodness

In [ ]:
import json

In [ ]:
with open("path_predictions.json","w") as f:
    f.write(json.dumps({str(p): float(f) for (p,f) in zip(pics, pic_goodness)}))

In [ ]:
manual_predictions = {}
with open("manual_predictions.json", "w") as f:
    f.write(json.dumps(manual_predictions))